# Display gather results w/taxonomy

In [ ]:
from charcoal import utils
import sourmash
from sourmash.lca import taxlist, LineagePair
import collections

import plotly.graph_objects as go

import sourmash_sankey
import importlib

import os.path

import screed

from sourmash.lca.command_index import load_taxonomy_assignments
from sourmash.lca import LCA_Database

from charcoal.lineage_db import LineageDB
from charcoal.utils import (gather_at_rank, get_ident, ContigGatherInfo)

In [ ]:
def do(lineages_csv, genome_filename, genome_sig_filename, matches_filename):
    genomebase = os.path.basename(genome_filename)
    match_rank = 'genus'

    # load taxonomy CSV                                                         
    tax_assign, _ = load_taxonomy_assignments(lineages_csv,
                                              start_column=3)
    print(f'loaded {len(tax_assign)} tax assignments.')

    # load the genome signature                                                 
    genome_sig = sourmash.load_one_signature(genome_sig_filename)

    # load all of the matches from search --containment in the database         
    with open(matches_filename, 'rt') as fp:
        try:
            siglist = list(sourmash.load_signatures(fp, do_raise=True,
                                                    quiet=False))
        except sourmash.exceptions.SourmashError:
            siglist = []
    print(f"loaded {len(siglist)} matches from '{matches_filename}'")

    # Hack for examining members of our search database: remove exact matches.  
    new_siglist = []
    for ss in siglist:
        if genome_sig.similarity(ss) == 1.0:
            print(f'removing an identical match: {ss.name()}')
        else:
            new_siglist.append(ss)
    siglist = new_siglist

    # if, after removing exact match(es), there is nothing left, quit.                                                                 
    if not siglist:
        print('no non-identical matches for this genome, exiting.')
        return {}

    # construct a template minhash object that we can use to create new 'uns    
    empty_mh = siglist[0].minhash.copy_and_clear()
    ksize = empty_mh.ksize
    scaled = empty_mh.scaled
    moltype = empty_mh.moltype

    # create empty LCA database to populate...                                  
    lca_db = LCA_Database(ksize=ksize, scaled=scaled, moltype=moltype)
    lin_db = LineageDB()

    # ...with specific matches.                                                 
    for ss in siglist:
        ident = get_ident(ss)
        lineage = tax_assign[ident]

        lca_db.insert(ss, ident=ident)
        lin_db.insert(ident, lineage)

    print(f'loaded {len(siglist)} signatures & created LCA Database')

    print('')
    print(f'reading contigs from {genomebase}')

    screed_iter = screed.open(genome_filename)
    contigs_tax = {}
    for n, record in enumerate(screed_iter):
       # look at each contig individually                                      
        mh = empty_mh.copy_and_clear()
        mh.add_sequence(record.sequence, force=True)

        # collect all the gather results at genus level, together w/counts;     
        # here, results is a list of (lineage, count) tuples.                   
        results = list(gather_at_rank(mh, lca_db, lin_db, match_rank))

        # store together with size of sequence.                                 
        info = ContigGatherInfo(len(record.sequence), len(mh), results)
        contigs_tax[record.name] = info

    print(f"Processed {len(contigs_tax)} contigs.")
    return contigs_tax


In [ ]:
importlib.reload(sourmash_sankey)
from sourmash_sankey import GenomeSankeyFlow

def make_sankey_fig(title, genome_lineage, contigs_info):
    obj = GenomeSankeyFlow()
    
    counts = collections.Counter()
    for contig_name, gather_info in contigs_info.items():
        contig_taxlist = gather_info.gather_tax

        # iterate over each contig match and summarize counts.              
        # note - here we can stop at first one, or track them all.          
        # note - b/c gather counts each hash only once, these               
        #     are non-overlapping                                           
        total_hashcount = 0
        for lin, hashcount in contig_taxlist:
            counts[lin] += hashcount
            total_hashcount += hashcount

        # track missing => unassigned lineage
        unident = gather_info.num_hashes - total_hashcount
        counts[obj.unassigned_lin] += unident
    
    # set the color of the main lineage
    genome_lineage = tuple(genome_lineage)
    obj.colors[genome_lineage] = "lightseagreen"
    
    # for phylum level disagreements, let's go with palevioletred
    for lin in counts:
        if not utils.is_lineage_match(lin, genome_lineage, 'phylum'):
            obj.colors[lin] = 'palevioletred'
            
    # assign unassigned to good lineage, maybe?
    counts[genome_lineage] += counts[obj.unassigned_lin]
    del counts[obj.unassigned_lin]
    
    obj.make_links(genome_lineage, counts)
    fig = obj.make_plotly_fig(title)
    
    return fig

genome_lin = 'd__Archaea,p__Thermoplasmatota,c__Poseidoniia,o__Poseidoniales,f__Thalassoarchaeaceae,g__MGIIb-O3,s__MGIIb-O3 sp002722735'
genome_lin = utils.make_lineage(genome_lin)

lineages_csv = '../charcoal/db/gtdb-release89-lineages.csv'

contigs_info = do(lineages_csv, 'tobg_np-110.fa', 'tobg_np-110.k31.sig', 'tobg_np-110.sig.matches')
make_sankey_fig('tobg_np-110', genome_lin, contigs_info)